In [2]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching comf_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s comf_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [62]:
# zentraler Stoß
m1 = 5
m2 = 7.5
m3 = 7.5

r1 = np.sqrt(m1/PI/10)
r2 = np.sqrt(m2/PI/10)
r3 = np.sqrt(m3/PI/10)


def x_i(T, dt):
    N = int(T/dt)

    x_array = np.zeros((N+1, 3))
    x_com_array = np.zeros((N+1, 3))
    com_array = np.zeros((N+1, ))

    x1 = -6
    x2 = -2
    x3 = 3

    v_x1 = 10
    v_x2 = 0    # FIX
    v_x3 = 0
    
    com = (m1*x1+m2*x2+m3*x3) / (m1+m2+m3)
    com_array[0] = com

    # Arrays der Positionen der 3 Massen im Labor- und Schwerpunktsystem
    x_array[0][0] = x1
    x_array[0][1] = x2
    x_array[0][2] = x3

    x_com_array[0][0] = x1 - com
    x_com_array[0][1] = x2 - com
    x_com_array[0][2] = x3 - com

    i = 1

    # Konstante Geschwindigkeit von m1 bis Treffen auf m2
    while (x1 + r1 < x2- r2):
        x1 += v_x1*dt

        com = (m1*x1+m2*x2+m3*x3) / (m1+m2+m3)
        com_array[i] = com

        x_array[i][0] = x1
        x_array[i][1] = x2
        x_array[i][2] = x3

        x_com_array[i][0] = x1 - com
        x_com_array[i][1] = x2 - com
        x_com_array[i][2] = x3 - com

        i += 1

    # Vollkommen elastischer Stoß von m1 auf m2
    v_x2 = v_x1 * 2*m1 / (m1 + m2)   
    v_x1 = v_x1 * (m1 - m2) / (m1 + m2)  

    # Konstante Geschwindigkeit m1 und m2 bis Treffen m2 auf m3
    while (x2 + r2 < x3- r3):
        x1 += v_x1*dt
        x2 += v_x2*dt

        com = (m1*x1+m2*x2+m3*x3) / (m1+m2+m3)
        com_array[i] = com

        x_array[i][0] = x1
        x_array[i][1] = x2
        x_array[i][2] = x3

        x_com_array[i][0] = x1 - com
        x_com_array[i][1] = x2 - com
        x_com_array[i][2] = x3 - com

        i += 1

    # Vollkommen inelastischer Stoß von m2 auf m3
    v_x2 = v_x2 * m2 / (m2 + m3)
    v_x3 = v_x2

    # Konstante Geschwindigkeit m1 und m2+m3 bis Ende des Arrays
    while (i <= N):
        x1 += v_x1*dt
        x2 += v_x2*dt
        x3 += v_x3*dt

        com = (m1*x1+m2*x2+m3*x3) / (m1+m2+m3)
        com_array[i] = com

        x_array[i][0] = x1
        x_array[i][1] = x2
        x_array[i][2] = x3

        x_com_array[i][0] = x1 - com
        x_com_array[i][1] = x2 - com
        x_com_array[i][2] = x3 - com

        i += 1
    
    return x_array, x_com_array, com_array

In [63]:
%%capture_video --path "animations/center_of_mass_force/central_push_F1.3.mp4"
%%manim -qk --fps 60 $video_scene

class comf_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Parameter
        T = 10
        dt = 0.01

        # Generierung der Datenarrays
        x_array, x_com_array, com_array = x_i(T, dt)
        com = [com_array[0], 0, 0]
        
        text1 = Text("(a) Laborsystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)
        text2 = Text("(b) Schwerpunktsystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)

        if m1 == m2 and m1 == m3:
            mass_text = MathTex(r'm_1 = m_2 = m_3', font_size = 36).align_on_border(DOWN + LEFT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(WHITE)
            mass_text[0][6:8].set_color(YELLOW)
        elif m1 == m2 and m1 < m3:
            mass_text = MathTex(r'm_1 = m_2 < m_3', font_size = 36).align_on_border(DOWN + LEFT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(WHITE)
            mass_text[0][6:8].set_color(YELLOW)
        elif m1 > m2 and m2 == m3:
            mass_text = MathTex(r'm_1 > m_2 = m_3', font_size = 36).align_on_border(DOWN + LEFT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(WHITE)
            mass_text[0][6:8].set_color(YELLOW)
        elif m1 < m2 and m2 == m3:
            mass_text = MathTex(r'm_1 < m_2 = m_3', font_size = 36).align_on_border(DOWN + LEFT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(WHITE)
            mass_text[0][6:8].set_color(YELLOW)
        elif m1 < m2 and m1 == m3:
            mass_text = MathTex(r'm_1 = m_3 < m_2', font_size = 36).align_on_border(DOWN + LEFT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(YELLOW)
            mass_text[0][6:8].set_color(WHITE)
        else:
            mass_text = MathTex(f'm_1: {m1},~m_2: {m2},~m_3: {m3}', font_size = 36).align_on_border(UP + RIGHT, buff = 0.5)
            mass_text[0][0:2].set_color(RED)
            mass_text[0][3:5].set_color(WHITE)
            mass_text[0][6:8].set_color(YELLOW)

        npla = NumberPlane(x_range = [-30, 30, 1], y_range  = [-5, 5, 1], x_length = 60, y_length = 10, axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25})
        npla2 = NumberPlane(x_range = [-30, 30, 1], y_range  = [-5, 5, 1], x_length = 60, y_length = 10, axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25})
        com_x_axis = Line([com_array[0], -5, 0], [com_array[0], 5, 0], color = WHITE, stroke_width = 0.5)
        com_x = Text("x (COM)", font_size = 16).move_to([com_array[0] + 0.75, 3.75, 0])
        

        # 3 Masses
        m1_dot = Dot(radius = r1, color = RED).move_to([x_array[0][0], 0, 0])
        m2_dot = Dot(radius = r2, color = WHITE).move_to([x_array[0][1], 0, 0])
        m3_dot = Dot(radius = r3, color = YELLOW).move_to([x_array[0][2], 0, 0])

        # Laborositionen als Iterator
        m1_dot.iter = iter(x_array[::,0])
        m2_dot.iter = iter(x_array[::,1])
        m3_dot.iter = iter(x_array[::,2])

        # COM-Positionen als Iterator
        m1_dot.com_iter = iter(x_com_array[::,0])
        m2_dot.com_iter = iter(x_com_array[::,1])
        m3_dot.com_iter = iter(x_com_array[::,2])

        com_x_axis.iter = iter(com_array)
        com_x.iter = iter(com_array)
        npla.iter = iter(com_array)

        def mass_updater(mass):
            mass.move_to([next(mass.iter), 0, 0])

        def com_x_updater(label):
            label.move_to([next(label.iter) + 0.75, 3.75, 0])

        def mass_com_updater(mass):
            mass.move_to([next(mass.com_iter), 0, 0])

        def npla_updater(cplane):
            cplane.move_to([-next(cplane.iter), 0, 0])


########## Animation ##########

        self.add(npla)
        self.play(Write(text1), run_time = 1.5)
        self.play(FadeIn(m1_dot), FadeIn(m2_dot), FadeIn(m3_dot), FadeIn(com_x_axis), FadeIn(com_x), FadeIn(mass_text), run_time = 3)
        self.play(Unwrite(text1), run_time = 1.5)
        self.wait(0.5)

        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)
        m3_dot.add_updater(mass_updater)
        com_x_axis.add_updater(mass_updater)
        com_x.add_updater(com_x_updater)

        timeline = ValueTracker(0)
        self.play(timeline.animate.set_value(5), rate_func= linear, run_time = 4)
        self.play(FadeOut(m1_dot), FadeOut(m2_dot), FadeOut(m3_dot), FadeOut(com_x_axis), FadeOut(com_x), run_time = 4.5)

        m1_dot.remove_updater(mass_updater)
        m2_dot.remove_updater(mass_updater)
        m3_dot.remove_updater(mass_updater)
        com_x_axis.remove_updater(mass_updater)
        com_x.remove_updater(com_x_updater)

        m1_dot.move_to([x_com_array[0][0], 0, 0])
        m2_dot.move_to([x_com_array[0][1], 0, 0])
        m3_dot.move_to([x_com_array[0][2], 0, 0])

        com_x_axis.move_to([0, 0, 0])
        com_x.move_to([0.75, 3.75, 0])

        self.play(Write(text2), ApplyMethod(npla.shift, com_array[0]*LEFT), run_time = 1.5)
        self.play(FadeIn(m1_dot), FadeIn(m2_dot), FadeIn(m3_dot), FadeIn(com_x_axis), FadeIn(com_x), run_time = 3)
        self.play(Unwrite(text2), run_time = 1.5)
        self.wait(0.5)

        m1_dot.add_updater(mass_com_updater)
        m2_dot.add_updater(mass_com_updater)
        m3_dot.add_updater(mass_com_updater)
        npla.add_updater(npla_updater)

        timeline = ValueTracker(0)
        self.play(timeline.animate.set_value(5), rate_func= linear, run_time = 4)
        self.play(FadeOut(m1_dot), FadeOut(m2_dot), FadeOut(m3_dot), FadeOut(com_x_axis), FadeOut(com_x), FadeOut(mass_text), run_time = 4.5)
        self.wait(3)
        #self.play(FadeTransform(npla, npla2), run_time = 1.5)

Output saved by creating file at animations/center_of_mass_force/central_push_F1.3.mp4.
